# Import

In [1]:
import pandas as pd
import numpy as np
import sqlite3

# Neo4J Queries

    - provider_nodes
        - organization_nodes
        - taxonomy_nodes
    - hospital_nodes
        - organization_nodes
        - taxonomy_nodes
    - referral_relationships

## provider_nodes
    - entity type 1
    - any cbsa
    - referred to an entity type 2 in cbsa 34980
        - with at least 50 transactions and
        - with less than 50 avg wait time

In [7]:
%%time
# referrals from providers (entity 1) to metro nashville groups (entity 2) 
# having over 50 transactions and under 50 average wait time
query = """
        
        WITH metro_entity_2 AS (
            SELECT *
            FROM profile
            INNER JOIN taxonomy
            ON profile.taxonomy_code = taxonomy.code
            WHERE provider_business_practice_location_address_postal_code IN (
                SELECT zip
                FROM zip_cbsa
                WHERE cbsa = '34980'
                )
                AND entity_type_code = 2.0
            ),
        referrals50 AS (
            SELECT *
            FROM referrals
            WHERE average_day_wait < 50
            AND transaction_count >= 50
        ),
        qualifying_refs AS (
            SELECT *
            FROM metro_entity_2
            INNER JOIN referral_to
            ON metro_entity_2.npi = referral_to.to_npi
            INNER JOIN referrals50
            USING (referral_id)
        ),
        entity1 AS (
            SELECT npi
            FROM profile
            INNER JOIN taxonomy
            ON profile.taxonomy_code = taxonomy.code
            WHERE entity_type_code = 1.0
        )
        SELECT 
            to_npi,
            referral_id,
            qf.entity_type_code,
            qf.taxonomy_code,
            classification,
            specialization,
            qf."provider_organization_name_(legal_business_name)" AS organization,
            qf.provider_first_line_business_practice_location_address AS address_line1,
            qf.provider_second_line_business_practice_location_address AS address_line2,
            qf.provider_business_practice_location_address_city_name AS city,
            qf.provider_business_practice_location_address_state_name AS state,
            qf.provider_business_practice_location_address_postal_code AS zip
        FROM qualifying_refs AS qf
        INNER JOIN referral_from
        USING (referral_id)
        INNER JOIN entity1
        ON referral_from.from_npi = entity1.npi;

"""
with sqlite3.connect('../data/hopteam.sqlite') as db:
    groups = pd.read_sql(query, db)

Wall time: 22.8 s


In [3]:
%%time
# profiles of all providers
# with referrals to metro nashville groups (entity 2) having
# referrals of over 50 transactions and under 50 average wait time
query = """
        
        WITH metro_entity_2 AS (
            SELECT *
            FROM profile
            WHERE provider_business_practice_location_address_postal_code IN (
                SELECT zip
                FROM zip_cbsa
                WHERE cbsa = '34980'
                )
                AND entity_type_code = 2.0
            ),
        referrals50 AS (
            SELECT *
            FROM referrals
            WHERE average_day_wait < 50
            AND transaction_count >= 50
        ),
        qualifying_refs AS (
            SELECT 
                referral_id
            FROM metro_entity_2
            INNER JOIN referral_to
            ON metro_entity_2.npi = referral_to.to_npi
            INNER JOIN referrals50
            USING (referral_id)
        ),
        entity1 AS (
            SELECT *
            FROM profile
            INNER JOIN taxonomy
            ON profile.taxonomy_code = taxonomy.code
            WHERE entity_type_code = 1.0
        )
        SELECT
            from_npi,
            referral_id,
            entity_type_code,
            "provider_last_name_(legal_name)" || ' ' || provider_first_name AS name,
            taxonomy_code,
            classification,
            specialization,
            "provider_organization_name_(legal_business_name)" AS organization,
            provider_first_line_business_practice_location_address AS address_line1,
            provider_second_line_business_practice_location_address AS address_line2,
            provider_business_practice_location_address_city_name AS city,
            provider_business_practice_location_address_state_name AS state,
            provider_business_practice_location_address_postal_code AS zip     
        FROM qualifying_refs
        INNER JOIN referral_from
        USING (referral_id)
        INNER JOIN entity1
        ON referral_from.from_npi = entity1.npi;

"""

with sqlite3.connect('../data/hopteam.sqlite') as db:
    providers = pd.read_sql(query, db)

Wall time: 23.8 s


In [8]:
%%time
# referrals from providers (entity 1) to metro nashville groups (entity 2) 
# having over 50 transactions and under 50 average wait time
query = """
        
        WITH metro_entity_2 AS (
            SELECT *
            FROM profile
            WHERE provider_business_practice_location_address_postal_code IN (
                SELECT zip
                FROM zip_cbsa
                WHERE cbsa = '34980'
                )
                AND entity_type_code = 2.0
            ),
        referrals50 AS (
            SELECT *
            FROM referrals
            WHERE average_day_wait < 50
            AND transaction_count >= 50
        ),
        qualifying_refs AS (
            SELECT
                to_npi,
                referral_id,
                patient_count,
                transaction_count,
                average_day_wait,
                std_day_wait  
            FROM metro_entity_2
            INNER JOIN referral_to
            ON metro_entity_2.npi = referral_to.to_npi
            INNER JOIN referrals50
            USING (referral_id)
        ),
        entity1 AS (
            SELECT *
            FROM profile
            INNER JOIN taxonomy
            ON profile.taxonomy_code = taxonomy.code
            WHERE entity_type_code = 1.0
        )
        SELECT
            from_npi,
            to_npi,
            referral_id,
            patient_count,
            transaction_count,
            average_day_wait,
            std_day_wait
        FROM qualifying_refs
        INNER JOIN referral_from
        USING (referral_id)
        INNER JOIN entity1
        ON referral_from.from_npi = entity1.npi;

"""

with sqlite3.connect('../data/hopteam.sqlite') as db:
    referrals = pd.read_sql(query, db)

Wall time: 23.9 s


## Explore the dfs

In [9]:
print('There are ', providers['from_npi'].nunique(), 'unique recommending providers.')
print('There are ', groups['organization'].nunique(), 'unique entity 2 organizations referred to.')


There are  17773 unique recommending providers.
There are  1262 unique entity 2 organizations referred to.


In [10]:
groups['organization'].nunique()

1262

In [14]:
(groups
     .loc[(groups['classification'].str.contains('General Acute Care Hospital', na = False)) 
           & 
           (groups['city'].str.contains('NASHVILLE'))
           ,
          ['organization', 'classification', 'specialization', 'zip']]
     .drop_duplicates()
     .reset_index()
)

,index,organization,classification,specialization,zip
0,33509,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,None,37203
1,43782,SAINT THOMAS WEST HOSPITAL,General Acute Care Hospital,None,37205
2,45127,HTI MEMORIAL HOSPITAL CORPORATION,General Acute Care Hospital,None,37207
3,46760,NASHVILLE GENERAL HOSPITAL,General Acute Care Hospital,None,37208
4,49937,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,None,37211
5,51105,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None,37212
6,59044,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None,37232
7,65635,SAINT THOMAS WEST HOSPITAL,General Acute Care Hospital,None,37236
8,66249,SETON CORPORATION,General Acute Care Hospital,None,37236


In [15]:
(groups
     .loc[(groups['classification'].str.contains('General Acute Care Hospital', na = False)) 
           & 
           (groups['city'].str.contains('NASHVILLE|MURFREESBORO|BRENTWOOD|FRANKLIN|HENDERSONVILLE|GOODLETTSVILLE|LA VERGNE|SMYRNA|MT. JULIET|GALLATIN'))
           ,
          ['organization', 'classification', 'specialization', 'city']]
     .drop_duplicates()
     .reset_index()
)

,index,organization,classification,specialization,city
0,9480,"SUMNER REGIONAL HEALTH SYSTEMS, INC",General Acute Care Hospital,None,GALLATIN
1,9756,SUMNER REGIONAL MEDICAL CENTER LLC,General Acute Care Hospital,None,GALLATIN
2,10861,WILLIAMSON COUNTY HOSPITAL DISTRICT,General Acute Care Hospital,None,FRANKLIN
3,14663,HENDERSONVILLE HOSPITAL CORPORATION,General Acute Care Hospital,None,HENDERSONVILLE
4,20701,"TRUSTPOINT HOSPITAL, LLC",General Acute Care Hospital,None,MURFREESBORO
5,22153,SAINT THOMAS RUTHERFORD HOSPITAL,General Acute Care Hospital,None,MURFREESBORO
6,24697,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,None,SMYRNA
7,33509,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,None,NASHVILLE
8,43782,SAINT THOMAS WEST HOSPITAL,General Acute Care Hospital,None,NASHVILLE
9,45127,HTI MEMORIAL HOSPITAL CORPORATION,General Acute Care Hospital,None,NASHVILLE


In [16]:
(groups
     .loc[groups['organization'].str.contains('VANDERBILT', na = False),
          ['organization', 'classification', 'specialization', 'zip']]
     .drop_duplicates()
)

,organization,classification,specialization,zip
8700,VANDERBILT HEALTH AND WILLIAMSON MEDICAL CENTE...,Internal Medicine,None,37064
18178,VANDERBILT UNIVERSITY MEDICAL CENTER,Psychiatric Unit,None,37087
18182,VANDERBILT UNIVERSITY MEDICAL CENTER,Rehabilitation Unit,None,37087
18185,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None,37087
26037,"VANDERBILT MAURY RADIATION ONCOLOGY, LLC",Clinic/Center,"Oncology, Radiation",37174
41143,VANDERBILT UNIVERSITY MEDICAL CENTER,Clinic/Center,End-Stage Renal Disease (ESRD) Treatment,37204
44951,"VANDERBILT IMAGING SERVICES, LLC",Clinic/Center,Radiology,37205
50958,VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP,Rehabilitation Hospital,None,37212
51038,"VANDERBILT IMAGING SERVICES, LLC",Radiology,Diagnostic Radiology,37212
51105,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None,37212


In [18]:
(groups
     .loc[groups['classification'].str.contains('Hospital'), 
          ['organization', 'classification', 'specialization']]
     .drop_duplicates()
     .sort_values('classification')
)

,organization,classification,specialization
185,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,Critical Access
45127,HTI MEMORIAL HOSPITAL CORPORATION,General Acute Care Hospital,None
43782,SAINT THOMAS WEST HOSPITAL,General Acute Care Hospital,None
26270,"SAINT THOMAS STONES RIVER HOSPITAL, LLC",General Acute Care Hospital,None
25460,NORTHCREST MEDICAL CENTER,General Acute Care Hospital,None
24615,"SAINT THOMAS DEKALB HOSPITAL, LLC",General Acute Care Hospital,None
24000,SHELBYVILLE HOSPITAL COMPANY LLC,General Acute Care Hospital,None
22153,SAINT THOMAS RUTHERFORD HOSPITAL,General Acute Care Hospital,None
20701,"TRUSTPOINT HOSPITAL, LLC",General Acute Care Hospital,None
66649,MAURY REGIONAL HOSPITAL,General Acute Care Hospital,None


In [ ]:
referrals